In [1]:
!pip install -q sentence_transformers

In [2]:
import glob
import os
import io
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import math
import itertools
import random
import csv

from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import nltk
from nltk.stem import WordNetLemmatizer

from sentence_transformers import SentenceTransformer, util

from itertools import islice
import seaborn as sns
from collections import Counter

In [3]:
inf_df = pd.read_csv('/kaggle/input/link-prediction-data-train-test/train-test/train.csv')
inf_df

,Unnamed: 0.1,Unnamed: 0,source_id,target_id,Y,source_year,source_authors,source_title,source_jn,source_abstract,...,target_title,target_jn,target_abstract,year_diff,common_authors,same_journal,title_similarity,abstract_similarity,title_embed_sim,abstract_embed_sim
0,0,0,9510123,9502114,1,1995,NaN,infinite number potential surrounding 2d black...,phys.lett.,found infinite number potential surrounding 2d...,...,stability analysis dilatonic black hole two di...,phys.lett.,explicitly show net number degree freedom two-...,0,0,1,0.135275,0.085267,0.587071,0.634117
1,1,1,9707075,9604178,1,1997,"l.e.ibanez,a.m.uranga",6 n 1 string vacuum duality,NaN,winter school duality mt sorak korea february ...,...,string orientifolds,nucl.phys.,construct several example compactification typ...,1,0,0,0.122320,0.056957,0.355650,0.563668
2,2,2,9312155,9506142,0,1993,"p. kleban,i. vassileva",conformal field theory hyperbolic geometry,phys.rev.lett.,examine correspondence conformal field theory ...,...,wightman function behaviour event horizon extr...,class.quant.grav.,reissner-nordstr om black hole result followin...,-1,0,0,0.000000,0.014844,0.462136,0.692309
3,3,3,9911255,302165,0,1999,"philip c. argyres,sophie pell",comparing instanton contribution exact result n 2,jhep,supersymmetric scale invariant theory discus g...,...,five-dimensional gauge theory quantum mechanic...,NaN,show dijkgraaf-vafa matrix model proposal exte...,-1,0,0,0.000000,0.040996,0.126891,0.580086
4,4,4,9701033,209076,0,1997,NaN,quantum gravitational measure three-geometries,phys.lett.,modification published gravitational measure a...,...,israel condition gauss-bonnet theory friedmann...,NaN,brane universe additional bulk field assuming ...,-1,0,0,0.000000,0.058933,0.174730,0.730622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615507,615507,615507,9704211,9311015,1,1997,NaN,collinear qcd model,NaN,non-perturbative method quantisation light-con...,...,glueball spectrum 1 1 dimensional model qcd,nucl.phys.,kresimir puhep1 princeton edu pupt-1427 iassns...,4,0,0,0.207828,0.024924,0.478591,0.587548
615508,615508,615508,9709133,9202062,0,1997,"j.audretsch,ulf.jasper,v.d.skarzhinsky",bremsstrahlung relativistic electron aharonov-...,phys.rev.,investigate scattering electron infinitely thi...,...,plumbing minimal area metric,NaN,study metric minimal area punctured riemann su...,5,0,0,0.000000,0.015097,0.177080,0.658379
615509,615509,615509,7141,9610152,0,2000,"hiroshi ishikawa,satoshi watamura",free field realization d-brane group manifold,jhep,construct boundary state d-brane su 2 group ma...,...,canonical approach string theory massive backg...,class.quant.grav.,method constructing canonical gauge invariant ...,4,0,0,0.068753,0.029369,0.370226,0.686230
615510,615510,615510,207232,111196,1,2002,"j. alex,re,n.e. mavromatos,sarben sarkar",induced n 2 composite supersymmetry 2 1 dimension,NaN,starting n 1 scalar supermultiplets 2 1 dimens...,...,emergent gauge boson,NaN,model bled slovenia july 17-27 2001 old idea p...,1,0,0,0.000000,0.000000,0.463030,0.450781


In [4]:
df = inf_df[inf_df['Y']==1]
df

,Unnamed: 0.1,Unnamed: 0,source_id,target_id,Y,source_year,source_authors,source_title,source_jn,source_abstract,...,target_title,target_jn,target_abstract,year_diff,common_authors,same_journal,title_similarity,abstract_similarity,title_embed_sim,abstract_embed_sim
0,0,0,9510123,9502114,1,1995,NaN,infinite number potential surrounding 2d black...,phys.lett.,found infinite number potential surrounding 2d...,...,stability analysis dilatonic black hole two di...,phys.lett.,explicitly show net number degree freedom two-...,0,0,1,0.135275,0.085267,0.587071,0.634117
1,1,1,9707075,9604178,1,1997,"l.e.ibanez,a.m.uranga",6 n 1 string vacuum duality,NaN,winter school duality mt sorak korea february ...,...,string orientifolds,nucl.phys.,construct several example compactification typ...,1,0,0,0.122320,0.056957,0.355650,0.563668
5,5,5,9710020,9709228,1,1997,NaN,n 4 4 2d supersymmetric gauge theory brane con...,phys.lett.,construct type ii brane configuration n 4 4 su...,...,two dimensional mirror symmetry m-theory,nucl.phys.,construct two dimensional gauge theory n 4 4 s...,0,0,0,0.000000,0.366701,0.443055,0.711183
6,6,6,9901042,9510135,1,1999,NaN,k-theory reality orientifolds,commun.math.phys.,use equivariant k-theory classify charge new p...,...,bound state string p branes,nucl.phys.,recent discovery explicit conformal field theo...,4,0,0,0.000000,0.108457,0.366435,0.648095
8,8,8,9705079,9702201,1,1997,"g. michaud,r. c. myers",hermitian d-brane solution,phys.rev.,low-energy background field solution describin...,...,lecture string duality,NaN,set lecture review recent development string t...,0,0,0,0.000000,0.013690,0.230498,0.512667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615503,615503,615503,9155,9908060,1,2000,"n.d. lambert,i. sachs",low energy dynamic non-bps branes,NaN,nonperturbative quantum effect 2000 paris talk...,...,non-supersymmetric d-brane configuration bose-...,jhep,open string spectrum spectrum open string vari...,1,0,0,0.000000,0.192305,0.561012,0.690089
615506,615506,615506,303120,204240,1,2003,"t. imai,y. kitazawa,y. takayama,d. tomino",quantum correction fuzzy sphere,NaN,investigate quantum correction non-commutative...,...,mean field approximation iib matrix model emer...,nucl.phys.,dimensional space-time univ symmetry factor gr...,1,0,0,0.000000,0.076948,0.376919,0.668469
615507,615507,615507,9704211,9311015,1,1997,NaN,collinear qcd model,NaN,non-perturbative method quantisation light-con...,...,glueball spectrum 1 1 dimensional model qcd,nucl.phys.,kresimir puhep1 princeton edu pupt-1427 iassns...,4,0,0,0.207828,0.024924,0.478591,0.587548
615510,615510,615510,207232,111196,1,2002,"j. alex,re,n.e. mavromatos,sarben sarkar",induced n 2 composite supersymmetry 2 1 dimension,NaN,starting n 1 scalar supermultiplets 2 1 dimens...,...,emergent gauge boson,NaN,model bled slovenia july 17-27 2001 old idea p...,1,0,0,0.000000,0.000000,0.463030,0.450781


In [5]:
G = nx.from_pandas_edgelist(df, 'source_id', 'target_id', create_using=nx.DiGraph())

In [6]:
citation_counts = dict(G.in_degree())

In [7]:
# Calculate centrality measures
degree_centrality = nx.degree_centrality(G)

In [8]:
# Calculate Betweenness Centrality
betweenness_centrality = nx.betweenness_centrality(G)

In [9]:
# Calculate Eigenvector Centrality
eigenvector_centrality = nx.eigenvector_centrality(G)

In [10]:
# Apply PageRank algorithm
pagerank_scores = nx.pagerank(G)

In [11]:
influential_papers = []
for paper_id in G.nodes():
    influence_score = (citation_counts.get(paper_id, 0) +
                       degree_centrality.get(paper_id, 0) +
                       betweenness_centrality.get(paper_id, 0) +
                       eigenvector_centrality.get(paper_id, 0) +
                       pagerank_scores.get(paper_id, 0))
    influential_papers.append((paper_id, influence_score))

influential_papers = sorted(influential_papers, key=lambda x: x[1], reverse=True)

print("Top 10 Influential Papers:")
for i, (paper_id, influence_score) in enumerate(influential_papers[:10], start=1):
    print(f"{i}. Paper ID: {paper_id}, Influence Score: {influence_score}")

Top 10 Influential Papers:
1. Paper ID: 9711200, Influence Score: 2294.1010321465214
2. Paper ID: 9802150, Influence Score: 1674.069093170943
3. Paper ID: 9802109, Influence Score: 1560.0649125876987
4. Paper ID: 9407087, Influence Score: 1229.2422783237844
5. Paper ID: 9610043, Influence Score: 1129.0623025115594
6. Paper ID: 9510017, Influence Score: 1100.1829004353592
7. Paper ID: 9908142, Influence Score: 1068.0479678010715
8. Paper ID: 9503124, Influence Score: 1057.2857856379894
9. Paper ID: 9906064, Influence Score: 970.050132033358
10. Paper ID: 9408099, Influence Score: 964.1244869169307
